In [1]:
%%capture
# !uv pip install -r ../requirements.txt # if using uv
!pip install -r ../requirements.txt 

In [2]:
# kill all connections to the database before running this notebook

import psycopg
import os 

with psycopg.connect(dbname='postgres', user=os.getenv('POSTGRES_USER'), password=os.getenv('POSTGRES_PASSWORD'), host='localhost') as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        cur.execute("""
            SELECT pg_terminate_backend(pid)
            FROM pg_stat_activity
            WHERE datname = 'test_db'
              AND pid <> pg_backend_pid();
        """)

In [3]:
import os
os.chdir('../..') # to be able to import 
import pandas as pd

from db_utils import (create_db, create_embeddings_table,
                      create_pgvector_extension, delete_db,
                      insert_data_into_table, pg_connection)

In [4]:
# create a database
DB_NAME = 'test_db'
create_db(db_name=DB_NAME)

In [5]:
# create the pgvector extension
create_pgvector_extension(db_name=DB_NAME)

INFO:root:pgvector extension created


In [6]:
# create the embeddings table
create_embeddings_table(DB_NAME)

INFO:root:Embeddings table 'pg_embeddings_test' created.


In [7]:
# viewing the embeddings table
TB_NAME = 'pg_embeddings_test'
CONN = pg_connection(db_name=DB_NAME)
pd.read_sql_query(f'SELECT * FROM {TB_NAME}', CONN)

/tmp/ipykernel_63485/146122205.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(f'SELECT * FROM {TB_NAME}', CONN)


,id,chunk,embedding


In [8]:
# inserting data in the embeddings table
text_chunks = ['hello', 'world', 'how', 'are', 'you']
text_embeddings = [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6], [0.7, 0.8], [0.9, 1.0]] # fake embeddings 
insert_data_into_table(DB_NAME, text_chunks, text_embeddings, TB_NAME)
pd.read_sql_query(f'SELECT * FROM {TB_NAME}', CONN)

INFO:root:Data inserted into table 'pg_embeddings_test'. Failed chunks: 0
/tmp/ipykernel_63485/3580995286.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(f'SELECT * FROM {TB_NAME}', CONN)


,id,chunk,embedding
0,0390b703-9307-4125-94df-b1fa0be443a2,hello,"[0.1,0.2]"
1,5d1f5e42-84ba-49e7-928a-e396a13ea73b,world,"[0.3,0.4]"
2,1c4f3c1a-b5ac-4c1b-b6c2-7d41e18e93e7,how,"[0.5,0.6]"
3,73c3d997-7741-4d29-915a-0293f748717c,are,"[0.7,0.8]"
4,bffa6e32-6e57-499c-884f-32699e2c8d70,you,"[0.9,1]"


In [9]:
# delete the database
CONN.close()
delete_db(DB_NAME)

INFO:root:Database test_db deleted
